In [2]:
import math
import torch
import gpytorch
import sys
from matplotlib import pyplot as plt
sys.path.append('../')
from LBFGS import FullBatchLBFGS

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [16]:
# import os
# import urllib.request
# from scipy.io import loadmat
# dataset = 'protein'
# if not os.path.isfile(f'../{dataset}.mat'):
#     print(f'Downloading \'{dataset}\' UCI dataset...')
#     urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1nRb8e7qooozXkNghC5eQS0JeywSXGX2S',
#                                f'../{dataset}.mat')

# data = torch.Tensor(loadmat(f'../{dataset}.mat')['data'])

In [17]:
# import numpy as np

# N = data.shape[0]
# # make train/val/test
# n_train = int(0.8 * N)
# train_x, train_y = data[:n_train, :-1], data[:n_train, -1]
# test_x, test_y = data[n_train:, :-1], data[n_train:, -1]

# # normalize features
# mean = train_x.mean(dim=-2, keepdim=True)
# std = train_x.std(dim=-2, keepdim=True) + 1e-6 # prevent dividing by 0
# train_x = (train_x - mean) / std
# test_x = (test_x - mean) / std

# # normalize labels
# mean, std = train_y.mean(),train_y.std()
# train_y = (train_y - mean) / std
# test_y = (test_y - mean) / std

# # make continguous
# train_x, train_y = train_x.contiguous(), train_y.contiguous()
# test_x, test_y = test_x.contiguous(), test_y.contiguous()

# output_device = torch.device('cuda:0')

# train_x, train_y = train_x.to(output_device), train_y.to(output_device)
# test_x, test_y = test_x.to(output_device), test_y.to(output_device)

In [6]:
n_devices = torch.cuda.device_count()
print('Planning to run on {} GPUs.'.format(n_devices))

Planning to run on 2 GPUs.


In [11]:
from dataset import ToyGraspDataset
from torch.utils.data import TensorDataset, DataLoader

# USING 2D FORK DATA ===============================
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
n = 10

# train classifier
train_dataset = ToyGraspDataset(root='data', name='sim2real_train', size=100, is_pos_label=True, device=device)
train_dataloader = DataLoader(train_dataset, batch_size=512, shuffle=True)

test_dataset = ToyGraspDataset(root='data', name='sim2real_test', size=n, is_pos_label=True, device=device)
test_dataloader = DataLoader(test_dataset, batch_size=512, shuffle=False)



# dataset = ToyGraspDataset(root='data', size=50)
train_x = torch.Tensor(train_dataset.data[:,:2])
train_y = torch.Tensor(train_dataset.data[:,2]).long()


output_device = torch.device('cuda:0')

train_x, train_y = train_x.to(output_device), train_y.to(output_device)
# test_x, test_y = test_x.to(output_device), test_y.to(output_device)

sim2real!!!


In [45]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, n_devices):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        base_covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

        self.covar_module = gpytorch.kernels.MultiDeviceKernel(
            base_covar_module, device_ids=range(n_devices),
            output_device=output_device
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def train(train_x,
          train_y,
          n_devices,
          output_device,
          checkpoint_size,
          preconditioner_size,
          n_training_iter,
):
    likelihood = gpytorch.likelihoods.GaussianLikelihood().to(output_device)
    model = ExactGPModel(train_x, train_y, likelihood, n_devices).to(output_device)
    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters
#     optimizer = FullBatchLBFGS(model.parameters(), lr=0.1)
    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)


    with gpytorch.beta_features.checkpoint_kernel(checkpoint_size), \
         gpytorch.settings.max_preconditioner_size(preconditioner_size):

#         def closure():
#             optimizer.zero_grad()
#             output = model(train_x)
#             loss = -mll(output, train_y).sum()
#             return loss

#         loss = closure()
#         loss.backward()
        for i in range(n_training_iter):
            
#             options = {'closure': closure, 'current_loss': loss, 'max_ls': 10}
#             print(options)
#             loss, _, _, _, _, _, _, fail = optimizer.step(options)
#             print("here")
            optimizer.zero_grad()
            output = model(train_x)
            loss = -mll(output, train_y).sum()
            loss.backward()
            
            optimizer.step()

            print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                i + 1, n_training_iter, loss.item(),
                model.covar_module.module.base_kernel.lengthscale.item(),
                model.likelihood.noise.item()
            ))

#             if fail:
#                 print('Convergence reached!')
#                 break

    print(f"Finished training on {train_x.size(0)} data points using {n_devices} GPUs.")
    return model, likelihood

In [46]:
model, likelihood = train(train_x, train_y,
                          n_devices=n_devices, output_device=output_device,
                          checkpoint_size=10000,
                          preconditioner_size=100,
                          n_training_iter=20)

Iter 1/20 - Loss: 1.009   lengthscale: 0.644   noise: 0.644
Iter 2/20 - Loss: 0.987   lengthscale: 0.598   noise: 0.598
Iter 3/20 - Loss: 0.908   lengthscale: 0.593   noise: 0.602
Iter 4/20 - Loss: 0.901   lengthscale: 0.568   noise: 0.627
Iter 5/20 - Loss: 0.924   lengthscale: 0.544   noise: 0.642


/home/user/grasp_pytorch/lib/python3.9/site-packages/gpytorch/utils/cholesky.py:38: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/user/grasp_pytorch/lib/python3.9/site-packages/gpytorch/utils/cholesky.py:38: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/user/grasp_pytorch/lib/python3.9/site-packages/gpytorch/utils/cholesky.py:38: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(


NotPSDError: Matrix not positive definite after repeatedly adding jitter up to 1.0e-04.